In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Data Acquisition

In [2]:
def load_university_town_data():
    
    university_towns = pd.read_table('university_towns.txt', header = None)
    university_towns.columns = ['RegionName']
    
    return university_towns

print(load_university_town_data().head(5)) 

                                        RegionName
0                                    Alabama[edit]
1                    Auburn (Auburn University)[1]
2           Florence (University of North Alabama)
3  Jacksonville (Jacksonville State University)[2]
4       Livingston (University of West Alabama)[2]


In [3]:
def load_gdp_data():
    
    gdp = pd.read_excel('gdplev.xls', skiprows = 219)
    gdp = gdp[['1999q4', 12323.3]]
    gdp.columns = ['Quarter','GDP']
    
    return gdp
               
print(load_gdp_data().head(5))    

  Quarter      GDP
0  2000q1  12359.1
1  2000q2  12592.5
2  2000q3  12607.7
3  2000q4  12679.3
4  2001q1  12643.3


In [4]:
def load_housing_data():
    
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv', header = 0)
   
    return housing_data

print(load_housing_data().iloc[0:5, 0:7])

   RegionID    RegionName State                           Metro    CountyName  \
0      6181      New York    NY                        New York        Queens   
1     12447   Los Angeles    CA  Los Angeles-Long Beach-Anaheim   Los Angeles   
2     17426       Chicago    IL                         Chicago          Cook   
3     13271  Philadelphia    PA                    Philadelphia  Philadelphia   
4     40326       Phoenix    AZ                         Phoenix      Maricopa   

   SizeRank   1996-04  
0         1       NaN  
1         2  155000.0  
2         3  109700.0  
3         4   50000.0  
4         5   87200.0  


In [5]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 
          'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 
          'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 
          'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 
          'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 
          'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 
          'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 
          'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 
          'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 
          'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 
          'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 
          'VA': 'Virginia'}

# Data Preparation

In [6]:
def get_list_of_university_towns():
    
    towns = load_university_town_data()
    current_state = ""
    def get_state(cell):
        if cell.endswith("[edit]"):
            global current_state
            current_state = cell[:-6]
            return cell[:-6]
        else:
            return current_state

    towns["State"] = towns["RegionName"].map(get_state)

    towns = towns[~towns["RegionName"].str.endswith("[edit]")]
    towns["RegionName"] = towns["RegionName"].map(lambda x:x.split("(")[0].strip())
    towns = towns.reindex(columns=["State","RegionName"]).reset_index(drop=True)
    towns.State = towns.State.map(dict(zip(states.values(),states.keys())))

    return towns
    
   
    
get_list_of_university_towns()  

,State,RegionName
0,AL,Auburn
1,AL,Florence
2,AL,Jacksonville
3,AL,Livingston
4,AL,Montevallo
...,...,...
512,WI,River Falls
513,WI,Stevens Point
514,WI,Waukesha
515,WI,Whitewater


In [7]:
def convert_housing_data_to_quarters():
    
    df = load_housing_data()
    df['State'].replace(states, inplace= True)
    df= df.set_index(["State","RegionName"])
    df = df.iloc[:,49:250]
    
    def quarters(col):
        if col.endswith(("01", "02", "03")):
            s = col[:4] + "q1"
        elif col.endswith(("04", "05", "06")):
            s = col[:4] + "q2"
        elif col.endswith(("07", "08", "09")):
            s = col[:4] + "q3"
        else:
            s = col[:4] + "q4"
        return s  
    housing = df.groupby(quarters, axis = 1).mean()
    housing = housing.sort_index()
    return housing

convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

                            2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  109433.333333   
        Ardmore

# Data Analysis

In [8]:
def get_recession_start():
    
    gdp = load_gdp_data()
    for i in range(0,gdp.shape[0]-1):
        if (gdp.iloc[i-2][1]> gdp.iloc[i-1][1]) and (gdp.iloc[i-1][1]> gdp.iloc[i][1]):
            startdate= gdp.iloc[i-3][0]
    return  startdate
    
get_recession_start()

'2008q3'

In [9]:
def get_recession_end():
    
    gdplev = load_gdp_data()
    start = get_recession_start()
    start_index = gdplev[gdplev['Quarter'] == start].index.tolist()[0]
    gdplev=gdplev.iloc[start_index:]
    for i in range(2, len(gdplev)):
        if (gdplev.iloc[i-2][1] < gdplev.iloc[i-1][1]) and (gdplev.iloc[i-1][1] < gdplev.iloc[i][1]):
            return gdplev.iloc[i][0]


get_recession_end()

'2009q4'

In [10]:
def get_recession_bottom():
    
    gdplev = load_gdp_data()
    start = get_recession_start()
    start_index = gdplev[gdplev['Quarter'] == start].index.tolist()[0]
    
    end= get_recession_end()
    end_index = gdplev[gdplev['Quarter'] == end].index.tolist()[0]
   
    gdp= gdplev.iloc[start_index:end_index+1]
    bottom = gdp['GDP'].min()
    bottom_index= gdp[gdp["GDP"]== bottom].index.tolist()[0]-start_index
    
    return gdp.iloc[bottom_index]['Quarter']
    
get_recession_bottom()

'2009q2'

In [11]:
def run_ttest():
   
    hd = convert_housing_data_to_quarters()
    hd = hd.loc[:,'2008q3':'2009q2']
    hd = hd.reset_index()
    hd['pricefluc'] = hd.apply(lambda x: (x['2008q3'] - x['2009q2'])/x['2008q3'],axis=1)


    univtown = get_list_of_university_towns()['RegionName']
    univtown = set(univtown)

    def check(name):
        #check if the town is a university towns or not.
        if name['RegionName'] in univtown:
            return 1
        else:
            return 0
    hd['condition'] = hd.apply(check,axis=1)


    Uprime = hd[hd['condition']==0].loc[:,'pricefluc'].dropna()
    U = hd[hd['condition']==1].loc[:,'pricefluc'].dropna()

    if Uprime.mean() < U.mean():
         better='non-university town'
    else:
         better='university town'
    p = list(ttest_ind(Uprime, U))[1]
    if p<0.01:
        final = (True,p,better)
    else:
        final = (False,p,better)
    return final
ans=run_ttest()
print(ans)

(True, 0.00036641601595526505, 'university town')
